In [1]:
# !pip install pandas
# !pip install plotly
# !pip install matplotlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Step 1 Import historical price data

Read in the data file

In [3]:
df = pd.read_csv("sakura_data.csv")
col = list(df.columns)
# df.head()

Read Marriott and Hilton's data into different Data frame

In [4]:
col_mar = col[:9]
col_hlt = col[-8:]
col_hlt.insert(0,"Date")
df_mar = pd.read_csv("sakura_data.csv",usecols=col_mar)
df_hlt = pd.read_csv("sakura_data.csv",usecols=col_hlt)
# print(df_mar.head())
# print(df_hlt.head())

## Step 2 Set parameters
x, threshold

In [5]:
x = 1
THRESHOLD = 1.5

## Step 3 Calculate Spread, Standard Deviation of spread and Long Term Mean

In [6]:
spread = np.log(df.mar_Close/df.hlt_Close**x)
# print(spread)
# print(type(spread))
window_size = 250
std_spread = pd.Series(spread[1:]).rolling(window_size).std(ddof=1)
std_spread = np.asarray(std_spread[window_size-1:-1])
# print(std_spread)
lt_mean_spread = pd.Series(spread[1:]).rolling(window_size).mean()
lt_mean_spread = np.asarray(lt_mean_spread[window_size-1:-1])
# print(lt_mean_spread)
# print(type(std_spread))
# lt_mean_spread

## Step 4 Calculate Entry Signal

In [ ]:
low = lt_mean_spread - THRESHOLD * std_spread
go_down = spread[:len(lt_mean_spread)] < low
high = lt_mean_spread + THRESHOLD * std_spread
go_up = spread[:len(lt_mean_spread)] > high
date = df_mar.Date[:len(lt_mean_spread)]
hlt_Open = df_hlt.hlt_Open[:len(lt_mean_spread)]
mar_Open = df_mar.mar_Open[:len(lt_mean_spread)]

mydf = pd.DataFrame(data = date)
mydf['Low'] = go_down
mydf['High'] = go_up
mydf['hlt_Open'] = hlt_Open
mydf['mar_Open'] = mar_Open
mydf = mydf.reset_index()  # make sure indexes pair with number of rows


In [ ]:
plt.figure(figsize=(20,15))
plt.plot(spread[:len(lt_mean_spread)])
plt.plot(low, linestyle='--')
plt.plot(high,linestyle='--')

## Step 5 Output as a Blotter
Reference: <a href="https://pypi.org/project/QTPyLib/" target="_blank">QTPyLib 1.5.83</a>.

In [ ]:
# !pip install blotter

In [ ]:

blotter = pd.DataFrame(
    columns=['date', 'trip', 'action', 'ticker', 'size',
             'price', 'status']
)
for index, row in mydf.iterrows():
    if row["Low"] == True:
        blotter.loc[len(blotter.index)] = [row["Date"], 'Open', 'BUY', 'MAR', 1, row["mar_Open"], 'FILLED']
        blotter.loc[len(blotter.index)] = [row["Date"], 'Open', 'SELL', 'HIT', 1, row["hlt_Open"], 'FILLED']
    elif row["High"] == True:
        blotter.loc[len(blotter.index)] = [row["Date"], 'Open', 'SELL', 'MAR', 1, row["mar_Open"], 'FILLED']
        blotter.loc[len(blotter.index)] = [row["Date"], 'Open', 'BUY', 'HIT', 1, row["hlt_Open"], 'FILLED']


In [12]:

blotter = pd.DataFrame(
    columns=['date', 'trip', 'action', 'ticker', 'size',
             'price', 'status']
)
for index, row in mydf.iterrows():
    if row["Low"] == True:
        blotter.loc[len(blotter.index)] = [row["Date"], 'Open', 'BUY', 'MAR', 1, row["mar_Open"], 'FILLED']
        blotter.loc[len(blotter.index)] = [row["Date"], 'Open', 'SELL', 'HIT', 1, row["hlt_Open"], 'FILLED']
    elif row["High"] == True:
        blotter.loc[len(blotter.index)] = [row["Date"], 'Open', 'SELL', 'MAR', 1, row["mar_Open"], 'FILLED']
        blotter.loc[len(blotter.index)] = [row["Date"], 'Open', 'BUY', 'HIT', 1, row["hlt_Open"], 'FILLED']


,date,trip,action,ticker,size,price,status
0,2022-01-03,Open,SELL,MAR,1,155.95,FILLED
1,2022-01-03,Open,BUY,HIT,1,165.07,FILLED
2,2021-12-31,Open,SELL,MAR,1,154.83,FILLED
3,2021-12-31,Open,BUY,HIT,1,164.85,FILLED
4,2021-12-30,Open,SELL,MAR,1,155.38,FILLED
...,...,...,...,...,...,...,...
683,2018-02-01,Open,SELL,HIT,1,146.60,FILLED
684,2018-01-31,Open,BUY,MAR,1,86.52,FILLED
685,2018-01-31,Open,SELL,HIT,1,147.42,FILLED
686,2018-01-30,Open,BUY,MAR,1,86.50,FILLED
